In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

H:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [17]:

def add_layer(inputs, in_size, out_size, activation_function=None):
    global Weights
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    #该层乘积之后输出
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre=sess.run(prediction,feed_dict={xs:v_xs,ys:v_ys})
    correct_prediction=tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))#返回矩阵，相等值为1
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))#cast转换格式
    result=sess.run(accuracy,feed_dict={xs:v_xs,})
    #不加ys也可以 ，虽然tensorflow是递归运算，但是accuracy关联correct_prediction，correct_prediction关联
    #y_pre和v_ys 即使y_pre和xs，ys有关，但是y_pre已经计算出来，所以实际上result只关联y_pre和v_ys，这两个都是不需要喂值的，
    #已经有实体，存在于tensorflow中的张量节点
    return result

In [20]:
xs = tf.placeholder(tf.float32, [None, 784]) # 28x28
ys = tf.placeholder(tf.float32, [None, 10])
prediction = add_layer(xs, 784, 10, activation_function=tf.nn.softmax)

#注意这是log 所以结果需要有负号
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                reduction_indices=[1])) # loss

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(2000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    for j in range(10):
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
    if i % 100 == 0:
            print(compute_accuracy(mnist.test.images, mnist.test.labels))
            #print(sess.run(Weights))


0.0895
0.8016
0.8314
0.8562
0.8644
0.8721
0.8636
0.8774
0.8845
0.8851
0.8862
0.8876
0.8936
0.8924
0.8937
0.8952
0.8968
0.8983
0.8971
0.9007


# 解释这里出现0.098的原因
是因为学习率太大，容易跑发散了！！！！

一定要注意，学习率太大就容易跑发散了

修改过程：本来莫烦的教程使用的是每次训练，选取100个样本，每个样本只训练一次，这样不容易跑发散，出现0.098的概率。

当时如果每次训练，选取100个样本，每个样本训练10次，权重就非常容易出现跑发散。结果出现正确率0.098，权重weight都变成nan，可以打印权重看一下！！

而且这样出现nan造成之后的训练无法回归到正常的权重